In [1]:
import os
os.chdir('E:/GitHubProjects/educational/mathematical_modeling/')
import glob
import sys
sys.path.insert(1, 'E:/GitHubProjects/educational/mathematical_modeling/lab2/')

import pandas as pd
import linear_regression as lin_reg
import numpy as np
import random


In [69]:
data = pd.read_csv('lab2/data/final_dataset.txt', sep=';').drop('Unnamed: 0', axis=1)
print(data)

    growth_total  divorce_total  marriage_total  LE_total    GDP  migration
0        -239568         669374           26149      67.5  14503     322392
1        -129091         669374           26456      69.0  29505     316460
2          -4252         644099           24193      69.0  27875     289857
3          24013         667969           24498      70.0  19192     301110
4          30336         693729           24586      70.0  20810     262929
5          32037         611644           23318      70.0  19838     255204
6          -2287         608334           19848      71.0  20017     269375
7        -135818         611435           20922      72.0  20133     219696
8        -224566         583941           17889      72.0  18909     215534
9        -317233         620728           19115      72.5  24223     204006
10       -702072         564702           15412      70.5  31999     209672
11      -1043341         644208           18424      69.5  29986     208059
12       -59

In [70]:
multifactor_linear_regression = lin_reg.LinearRegressionAnalysis(data.drop('migration', axis=1), 'growth_total')

In [71]:
# Выполнение анализа на мультиколлинеарность
print("Multicollinearity check:")
result_df_X, result_df_df, corr_matrix, true_labels = multifactor_linear_regression.feature_analysis()
print("\nМатрица факторов X:\n",result_df_X)
print("\nМатрица факторов с таргетом:\n",result_df_df)
print("\nОтобранные переменные:\n",true_labels)

Multicollinearity check:

Матрица факторов X:
           feature  multicollinearity  tightness response
0   divorce_total              False      False        +
1  marriage_total              False      False        +
2        LE_total              False      False        -
3             GDP              False      False        -

Матрица факторов с таргетом:
                  feature  multicollinearity tightness response
0   target_divorce_total              False      None     None
1  target_marriage_total              False      None     None
2        target_LE_total              False      None     None
3             target_GDP              False      None     None

Отобранные переменные:
 ['divorce_total' 'marriage_total' 'LE_total' 'GDP']


In [72]:
data_with_lags_X1 = multifactor_linear_regression.add_lag_to_feature(feature_name='divorce_total', max_lag=2)
print("\nДобавление лага для divorce_total:")
print(data_with_lags_X1.head())


Добавление лага для divorce_total:
   growth_total  divorce_total  marriage_total  LE_total    GDP  \
0         -4252         644099           24193      69.0  27875   
1         24013         667969           24498      70.0  19192   
2         30336         693729           24586      70.0  20810   
3         32037         611644           23318      70.0  19838   
4         -2287         608334           19848      71.0  20017   

   divorce_total_lag1  divorce_total_lag2  
0            669374.0            669374.0  
1            644099.0            669374.0  
2            667969.0            644099.0  
3            693729.0            667969.0  
4            611644.0            693729.0  


In [73]:
# Выполнение анализа на мультиколлинеарность
print("Multicollinearity check:")
result_df_X, result_df_df, corr_matrix, true_labels = multifactor_linear_regression.feature_analysis()
print("\nМатрица факторов X:\n",result_df_X)
print("\nМатрица факторов с таргетом:\n",result_df_df)
print("\nОтобранные переменные:\n",true_labels)

Multicollinearity check:

Матрица факторов X:
               feature  multicollinearity  tightness response
0       divorce_total              False      False        +
1      marriage_total              False      False        +
2            LE_total              False      False        -
3                 GDP               True      False        -
4  divorce_total_lag1              False      False        +
5  divorce_total_lag2              False      False        +

Матрица факторов с таргетом:
                      feature  multicollinearity tightness response
0       target_divorce_total              False      None     None
1      target_marriage_total              False      None     None
2            target_LE_total              False      None     None
3                 target_GDP               True      None     None
4  target_divorce_total_lag1              False      None     None
5  target_divorce_total_lag2              False      None     None

Отобранные переменные:
 [

In [74]:
multifactor_linear_regression.estimate_linear_regression(true_labels)
print("\nModel Summary:")
print(multifactor_linear_regression.get_summary())


Model Summary:
                            OLS Regression Results                            
Dep. Variable:           growth_total   R-squared:                       0.916
Model:                            OLS   Adj. R-squared:                  0.846
Method:                 Least Squares   F-statistic:                     13.04
Date:                Wed, 11 Dec 2024   Prob (F-statistic):            0.00356
Time:                        21:12:08   Log-Likelihood:                -154.96
No. Observations:                  12   AIC:                             321.9
Df Residuals:                       6   BIC:                             324.8
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const              -

e:\GitHubProjects\educational\mathematical_modeling\base_venv10\lib\site-packages\scipy\stats\_axis_nan_policy.py:418: UserWarning: `kurtosistest` p-value may be inaccurate with fewer than 20 observations; only n=12 observations were given.
  return hypotest_fun_in(*args, **kwds)


In [75]:
print("\nModel Evaluation:")
evaluation_metrics = multifactor_linear_regression.evaluate_model()
for k,v in evaluation_metrics.items():
    print(f"{k}={np.round(v,2)}")


Model Evaluation:
MSE=9634069092.19
RMSE=98153.29
Relative Error=3.27
F-statistic=13.04
R-squared=0.92
Student t-test=[-3.73 -3.92  5.02  3.36  0.89  2.48]


In [82]:
# Пример предсказания для новых данных
new_data = pd.DataFrame({
    factor: [np.mean(data[factor])] for factor in true_labels
})
print("\nPrediction for new data:")
print(multifactor_linear_regression.predict(new_data))



Prediction for new data:


KeyError: "None of [Index(['divorce_total', 'marriage_total', 'LE_total', 'divorce_total_lag1',\n       'divorce_total_lag2'],\n      dtype='object')] are in the [columns]"

In [76]:
data_new = multifactor_linear_regression.extract_consecutive_range(4,11)

In [81]:
data_new

,growth_total,divorce_total,marriage_total,LE_total,GDP,divorce_total_lag1,divorce_total_lag2
3,32037,611644,23318,70.0,19838,NaN,NaN
4,-2287,608334,19848,71.0,20017,611644.0,NaN
5,-135818,611435,20922,72.0,20133,608334.0,611644.0
6,-224566,583941,17889,72.0,18909,611435.0,608334.0
7,-317233,620728,19115,72.5,24223,583941.0,611435.0
8,-702072,564702,15412,70.5,31999,620728.0,583941.0
9,-1043341,644208,18424,69.5,29986,564702.0,620728.0
10,-594557,682850,21271,72.0,38507,644208.0,564702.0


In [77]:
multifactor_linear_regression1 = lin_reg.LinearRegressionAnalysis(data_new, 'growth_total')
# Выполнение анализа на мультиколлинеарность
print("Multicollinearity check:")
result_df_X, result_df_df, corr_matrix, true_labels = multifactor_linear_regression1.feature_analysis()
print("\nМатрица факторов X:\n",result_df_X)
print("\nМатрица факторов с таргетом:\n",result_df_df)
print("\nОтобранные переменные:\n",true_labels)
data_with_lags_X1 = multifactor_linear_regression1.add_lag_to_feature(feature_name='divorce_total', max_lag=2)
print("\nДобавление лага для divorce_total:")
print(data_with_lags_X1.head())
# Выполнение анализа на мультиколлинеарность
print("Multicollinearity check:")
result_df_X, result_df_df, corr_matrix, true_labels = multifactor_linear_regression1.feature_analysis()
print("\nМатрица факторов X:\n",result_df_X)
print("\nМатрица факторов с таргетом:\n",result_df_df)
print("\nОтобранные переменные:\n",true_labels)
multifactor_linear_regression1.estimate_linear_regression(true_labels)
print("\nModel Summary:")
print(multifactor_linear_regression1.get_summary())

Multicollinearity check:

Матрица факторов X:
               feature  multicollinearity  tightness response
0       divorce_total              False      False        -
1      marriage_total              False      False        +
2            LE_total              False      False        +
3                 GDP               True      False        -
4  divorce_total_lag1              False      False        +
5  divorce_total_lag2               True      False        +

Матрица факторов с таргетом:
                      feature  multicollinearity tightness response
0       target_divorce_total              False      None     None
1      target_marriage_total              False      None     None
2            target_LE_total              False      None     None
3                 target_GDP               True      None     None
4  target_divorce_total_lag1              False      None     None
5  target_divorce_total_lag2              False      None     None

Отобранные переменные:
 [

E:\GitHubProjects/educational/mathematical_modeling/lab2\linear_regression.py:143: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data[f'{feature_name}_lag{lag}'] = self.data[feature_name].shift(lag)
e:\GitHubProjects\educational\mathematical_modeling\base_venv10\lib\site-packages\statsmodels\stats\stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 6 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "


In [79]:
print("\nModel Evaluation:")
evaluation_metrics = multifactor_linear_regression.evaluate_model()
for k,v in evaluation_metrics.items():
    print(f"{k}={np.round(v,2)}")


Model Evaluation:
MSE=8984804642.14
RMSE=94788.21
Relative Error=3.7
F-statistic=13.04
R-squared=0.92
Student t-test=[-3.73 -3.92  5.02  3.36  0.89  2.48]



Model Evaluation:
MSE=8984804642.14
RMSE=94788.21
Relative Error=3.7
F-statistic=13.04
R-squared=0.92
Student t-test=[-3.73 -3.92  5.02  3.36  0.89  2.48]
